In [1]:
import pandas as pd
import re 

'''def clean_company_name(name: str) -> str:
    """Normalize stock names to plain company names."""
    name = re.sub(r'\bCommon Stock\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bOrdinary Shares?\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bDepositary Shares?.*', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bClass [A-Z]\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bLtd\b', 'Limited', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+', ' ', name).strip()
    return name'''

# Load both ticker lists
#nyse_df = pd.read_csv("nyse_screener.csv")
#nasdaq_df = pd.read_csv("nasdaq_screener.csv")
Top_100 = pd.read_csv(r"C:\Users\lucas\OneDrive\Desktop\NLP\top_100_companies_2022.csv", encoding='latin1')

# Combine into a single DataFrame
#all_tickers = pd.concat([nyse_df, nasdaq_df], ignore_index=True)

# Normalize column names just in case
#all_tickers.columns = [col.strip().lower() for col in all_tickers.columns]
Top_100.columns = [col.strip().lower() for col in Top_100.columns]

# Clean names
#all_tickers['clean_name'] = all_tickers['name'].apply(clean_company_name)

# Create dictionaries for fast lookups
ticker_to_name = dict(zip(Top_100['symbol'].str.upper(), Top_100['name']))
valid_tickers = set(ticker_to_name.keys())
company_names = [name.lower() for name in ticker_to_name.values()]
name_to_ticker = {name.lower(): symbol for symbol, name in ticker_to_name.items()}

In [5]:
#imports
import json
import pandas as pd
import spacy
import re
from tqdm import tqdm
import spacy
import pandas as pd
import spacy
import warnings
from rapidfuzz import process, fuzz

#model
nlp = spacy.load('en_core_web_sm')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

#to be ran per comment
def extract_ner_entities(model, text, similarity_threshold=90):
    
    BLACKLIST = {'ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda', 'treasury', 'covid-19', 'rrsp', 'tfsa','fed', 'Reuters aggregated](https://www.streetinsider.com/Reuters'.lower()}
    doc = model(text)
    detected_companies = []

    #Detect companies via spaCy NER
    for ent in doc.ents:
        if ent.label_ == "ORG" and ent.text.lower() not in BLACKLIST:
            org_name = ent.text.strip()
            # Fuzzy match against official company names from csv file
            match, score, _ = process.extractOne(org_name.lower(), company_names, scorer=fuzz.token_sort_ratio)
            if score >= similarity_threshold:
                matched_ticker = name_to_ticker[match]
                canonical_name = ticker_to_name[matched_ticker]
                detected_companies.append(canonical_name)
            else:
                #keeps companies not in csv file maybe delete later
                detected_companies.append(org_name)

    # --- Match stock tickers in text ---
    for token in doc:
        token_text = token.text.strip()

        # Handle tickers with $ prefix, e.g. $AAPL
        if token_text.startswith("$"):
            token_text = token_text[1:]

        # Check if it’s a valid ticker symbol
        if token_text in valid_tickers:
            company_name = ticker_to_name.get(token_text)
            detected_companies.append(company_name)


    return list(set(detected_companies))


def get_dict_top_companies(dataset, column_name, top_companies=3):
    company_counter = dict()
    for companies in dataset[column_name]:
        for company in companies:
            if company in company_counter:  
                company_counter[company] += 1
            else:
                company_counter[company] = 1
    sorted_dict = dict(sorted(company_counter.items(), key=lambda x: x[1], reverse=True))
    top = dict()
    for company, count in list(sorted_dict.items())[:top_companies]:
        top[company] = count

    return top


In [ ]:
pre_processed_sub = pd.read_csv(r"C:\Users\lucas\OneDrive\Desktop\NLP\processed_sub_2.csv")
pre_processed_com = pd.read_csv(r"C:\Users\lucas\OneDrive\Desktop\NLP\processed_com.csv")

# Ensure every cell is a string (NaN -> "")
pre_processed_sub["selftext"] = pre_processed_sub["selftext"].fillna("").astype(str)
pre_processed_com["selftext"] = pre_processed_com["selftext"].fillna("").astype(str)

# Safe wrapper so extract_ner_entities always receives a string
def safe_extract(text):
    if not text or not isinstance(text, str):
        return []
    return extract_ner_entities(nlp, text, similarity_threshold=60)

pre_processed_sub["Companies"] = pre_processed_sub["selftext"].apply(safe_extract)
pre_processed_com["Companies"] = pre_processed_com["selftext"].apply(safe_extract)


{'Apple': 320, 'Tesla': 281, 'Amazon': 274}


In [ ]:
# --- Submissions ---
import os


top_dict_sub = get_dict_top_companies(pre_processed_sub, "Companies")
top_set_sub = set(top_dict_sub.keys())
print(f"Top companies: {sorted(top_set_sub)}")

mask_sub = pre_processed_sub['Companies'].apply(lambda lst: bool(top_set_sub.intersection(lst)))
filtered_df = pre_processed_sub[mask_sub].copy()

exploded_sub = filtered_df.explode('Companies')

# Keep only rows for top companies 
exploded_sub = exploded_sub[exploded_sub['Companies'].isin(top_set_sub)].copy()

# Create separate dataframes for each top company 
dfs_by_company_sub = {}
os.makedirs("companies_csv", exist_ok=True)

for company in top_set_sub:
    dfs_by_company_sub[company] = exploded_sub[exploded_sub['Companies'] == company].copy()
    file_path = os.path.join("companies_csv", f"{company}_submissions.csv")
    dfs_by_company_sub[company].to_csv(file_path, index=False)
    print(f"{company}: {len(dfs_by_company_sub[company])} rows saved to {file_path}")

print("\nPreview of each company's SUBMISSIONS dataframe:\n")
for company, df_company in dfs_by_company_sub.items():
    print(f"=== {company.upper()} ({len(df_company)} rows) ===")
    display(df_company.head(3))   
    print("\n")

filtered_df.to_csv("filtered_submissions_top_companies.csv", index=False)  #not really needed, just the dataframe with all top companies submissions
print("Saved filtered_submissions_top_companies.csv")


In [ ]:
# --- Comments ---
import os


top_dict_com = get_dict_top_companies(pre_processed_com, "Companies")
top_set_com = set(top_dict_com.keys())
print(f"Top companies (comments): {sorted(top_set_com)}")

mask_com = pre_processed_com['Companies'].apply(lambda lst: bool(top_set_com.intersection(lst)))
filtered_comments = pre_processed_com[mask_com].copy()

print(f"Comment rows before: {len(pre_processed_com)} | after filtering: {len(filtered_comments)}")

exploded_com = filtered_comments.explode('Companies')

exploded_com = exploded_com[exploded_com['Companies'].isin(top_set_com)].copy()

dfs_by_company_com = {}
os.makedirs("companies_csv_comments", exist_ok=True)

for company in top_set_com:
    dfs_by_company_com[company] = exploded_com[exploded_com['Companies'] == company].copy()
    file_path = os.path.join("companies_csv_comments", f"{company}_comments.csv")
    dfs_by_company_com[company].to_csv(file_path, index=False)
    print(f"{company}: {len(dfs_by_company_com[company])} comment rows saved to {file_path}")

print("\nPreview of each company's COMMENTS dataframe:\n")
for company, df_company in dfs_by_company_com.items():
    print(f"=== {company.upper()} ({len(df_company)} rows) ===")
    display(df_company.head(3))
    print()

filtered_comments.to_csv("filtered_comments_top_companies.csv", index=False)
print("Saved filtered_comments_top_companies.csv")
